### 회전 추가

In [ ]:
import os
import numpy as np
from PIL import Image, ImageOps
import pandas as pd
from tqdm import tqdm

import albumentations as A
import matplotlib.pyplot as plt 
from glob import glob 

import cv2

In [11]:
df = pd.read_csv('/root/CV_PJT/CV_PJT/data/data/train.csv')

In [ ]:
ids = []
targets = []
data_path = '/root/CV_PJT/CV_PJT/data/data/train'

def save_rotated_image(image_path, angle, prefix, ID, target):
    rotated_id = f'{prefix}_{ID}'
    ids.append(rotated_id)
    targets.append(target)
    rotated_image = np.array(ImageOps.exif_transpose(Image.open(image_path).rotate(angle, expand=True)))
    Image.fromarray(rotated_image).save(os.path.join(data_path, rotated_id))

for index, ID, target in tqdm(df.itertuples(), desc='Image rotation', mininterval=0.1):
    image_path = os.path.join(data_path, ID)
    
    # Rotate 90, 180, and 270 degrees
    # save_rotated_image(image_path, 90, 'r90', ID, target)
    # save_rotated_image(image_path, 180, 'r180', ID, target)
    # save_rotated_image(image_path, 270, 'r270', ID, target)
    
    # Rotate 45, 90, 135, 180, 225, and 270 degrees
    save_rotated_image(image_path, 45, 'r45', ID, target)
    save_rotated_image(image_path, 90, 'r90', ID, target)
    save_rotated_image(image_path, 135, 'r135', ID, target)
    save_rotated_image(image_path, 180, 'r180', ID, target)
    save_rotated_image(image_path, 225, 'r225', ID, target)
    save_rotated_image(image_path, 270, 'r270', ID, target)

rotate_data = {
    'ID': ids,
    'target': targets
}
rotate_df = pd.DataFrame(rotate_data)    
df = pd.concat([df, rotate_df])

Image rotation: 1570it [02:21, 11.10it/s]


In [13]:
h_flip = A.HorizontalFlip(always_apply=True, p=1)
v_flip = A.VerticalFlip(always_apply=True, p=1)

In [14]:
ids = []
targets = []
data_path = '/root/CV_PJT/CV_PJT/data/data/train'

def save_flipped_image(image, flip_transform, prefix, ID, target):
    flipped_id = f'{prefix}_{ID}'
    ids.append(flipped_id)
    targets.append(target)
    flipped_image = flip_transform(image=image)['image']
    Image.fromarray(flipped_image).save(os.path.join(data_path, flipped_id))

for index, ID, target in tqdm(df.itertuples(), desc='Image flip', mininterval=0.1):
    image_path = os.path.join(data_path, ID)
    image = np.array(Image.open(image_path))

    # Vertical flip for r90 or r270, Horizontal flip otherwise
    if ID.startswith('r90') or ID.startswith('r270'):
        save_flipped_image(image, v_flip, 'Vflip', ID, target)
    else:
        save_flipped_image(image, h_flip, 'Hflip', ID, target)

# Dataframe for flipped images
flip_data = {
    'ID': ids,
    'target': targets
}
flip_df = pd.DataFrame(flip_data)    
df = pd.concat([df, flip_df])

Image flip: 15700it [02:26, 107.50it/s]


In [22]:
transforms = A.Compose([
    A.OneOf([
        A.GridDistortion(num_steps=5, distort_limit=0.3, interpolation=1, border_mode=4, value=None, mask_value=None, always_apply=True, p=1),
        A.ElasticTransform(always_apply=True, p=1, alpha=1.0, sigma=50.0, alpha_affine=50.0, interpolation=0, border_mode=1, value=(0, 0, 0), mask_value=None, approximate=False),
        A.OpticalDistortion(always_apply=True, p=1, distort_limit=(-0.3, -0.1)),
        A.OpticalDistortion(always_apply=True, p=1, distort_limit=(0.1, 0.3)),
    ], p=0.85),
    A.SomeOf([
        A.RandomBrightnessContrast(brightness_limit=(-0.2, 0.2), contrast_limit=(-0.2, 0.2), p=1),
        A.HueSaturationValue(hue_shift_limit=15, sat_shift_limit=25, val_shift_limit=20, p=1),
        A.MultiplicativeNoise(p=1, multiplier=(1, 1.5), per_channel=True),
        A.Equalize(p=1, mode='cv', by_channels=True),
    ], n=2, p=0.85),
    A.OneOf([
        A.Rotate(limit=(10, 30), border_mode=cv2.BORDER_CONSTANT, p=1),
        A.Rotate(limit=(150, 170), border_mode=cv2.BORDER_CONSTANT, p=1),
        A.Rotate(limit=(190, 210), border_mode=cv2.BORDER_CONSTANT, p=1),
        A.Rotate(limit=(330, 350), border_mode=cv2.BORDER_CONSTANT, p=1),
    ], p=1),
    A.CoarseDropout(p=0.5, max_holes=40, max_height=15, max_width=15, min_holes=8, min_height=8, min_width=8),
    A.Equalize(p=0.5, mode='cv', by_channels=True),
    A.OneOf([
        A.Blur(blur_limit=(3, 4), p=1),
        A.MotionBlur(blur_limit=(3, 5), p=1),
        A.Downscale(scale_min=0.455, scale_max=0.5, interpolation=2, p=1),
    ], p=0.5),
    A.GaussNoise(var_limit=(100, 800), per_channel=True, p=0.5),
])

In [23]:
# Define augmentation count based on target
def get_augmentation_count(target):
    if target == 13:
        return 15
    elif target == 14:
        return 20
    elif target == 1:
        return 20
    else:
        return 10

ids = []
targets = []
data_path = '/root/CV_PJT/CV_PJT/data/data/train'

for index, ID, target in tqdm(df.itertuples(), desc='Image augmentation', mininterval=0.1):
    image_path = os.path.join(data_path, ID)
    image = np.array(Image.open(image_path))
    n = get_augmentation_count(target)
    
    for i in range(n):
        transformed_image = transforms(image=image)['image']
        image_ID = f'tf{i}_{ID}' 
        ids.append(image_ID)
        targets.append(target)
        Image.fromarray(transformed_image).save(os.path.join(data_path, image_ID))
    
# Create augmented DataFrame and concatenate with original
aug_data = {
    'ID': ids,
    'target': targets
}
aug_df = pd.DataFrame(aug_data)
df = pd.concat([df, aug_df])

Image augmentation: 31400it [6:46:46,  1.29it/s]


In [24]:
df.to_csv('/root/CV_PJT/CV_PJT/trin4.csv', index=False)